In [4]:
import pandas as pd

df_riket = pd.read_excel("../Laboration2/Data/riket2023_åk9_np.xlsx", sheet_name=None, skiprows=8)
sheets = "Engelska", "Matematik", "Svenska", "Svenska som andraspråk"

df_riket

{'Engelska':    Riket Typ av huvudman  Totalt  Flickor  Pojkar  Totalt   Flickor   Pojkar   \
 0  Riket        Samtliga  106941    51864   55077     97.0      97.1     97.0   
 1  Riket        Kommunal   84310    40675   43635     96.6      96.6     96.6   
 2  Riket         Enskild   22582    11165   11417     98.7      98.8     98.7   
 3  Riket      Skolverket      49       24      25    100.0     100.0    100.0   
 
    Totalt    Flickor    Pojkar    
 0      15.7       15.8      15.7  
 1      15.4       15.5      15.4  
 2      16.9       17.0      16.8  
 3      18.5       18.9      18.2  ,
 'Matematik':    Riket Typ av huvudman  Totalt  Flickor  Pojkar Totalt  Flickor  Pojkar   \
 0  Riket        Samtliga  106766    51548   55218    89.1     88.7    89.5   
 1  Riket        Kommunal   84163    40386   43777    88.1     87.7    88.5   
 2  Riket         Enskild   22553    11137   11416    92.9     92.2    93.5   
 3  Riket      Skolverket      50       25      25    ~100     ~10